# Here you test something

Cols meaning:
- Day - day of the month
- DKind - day kind
- EC - exercise count
- EI - exercise intensity - has six levels from 0 - didn't exercise to 5 - hard exercise (More info in a separate message)
- OD - outdoor
- LE - left-eye reading
- JG - juggling
- GM - gaming


(Separate message about EI)
Exercise intensity notation:
- 0 - you didn't exercise
- 1 - easy
- 2 - between easy and moderate
- 3 - moderate
- 4 - between moderate and hard
- 5 - hard

In [1]:
import numpy as np
import pandas as pd

from preparation.prepare_data import prepare_data
from constants import PATH, APPEND_PATH, PREVIOUS_DATA_FOLDER

DAY_OFFS = []
DESIRED_MEAN_VALUE = 275
report_data, wide_use_data = prepare_data(PATH, DAY_OFFS,
                                              DESIRED_MEAN_VALUE)
month, mean, std, min_to_study, df, EI_last = wide_use_data
df

,Day,DKind,Math,CS,Eng,Total,EC,EI,OD,LE,JG,GM
0,1,work,0,212,11,223,1,2,120,0,0,0
1,2,work,90,0,9,99,1,0,12,0,15,0


In [2]:
report_data

([2, 4, 0, 5, 161, 0, 'you studied every day'],
 [['OD', 'LE', 'JG', 'GM'],
  1,
  [2.2, 0.0, 0.25, 0.0],
  [2, 0, 1, 0],
  {1: 0, 2: 1, 3: 0, 4: 0, 5: 0}])

In [3]:
# somehow you need to figure out how to count how many days
# passed since you last had exercise of a particular intensity
# also, count how many days passed since the last time you exercised
nonstudy_df = df[["Day", "EC", "EI", "OD", "LE", "JG", "GM"]]
nonstudy_df

,Day,EC,EI,OD,LE,JG,GM
0,1,1,2,120,0,0,0
1,2,1,0,12,0,15,0


In [4]:
EI_last

{1: 100, 2: 1, 3: 100, 4: 100, 5: 100}

In [5]:
# current_day = nonstudy_df["Day"].iloc[-1]
cur_day = df["Day"].iloc[-1]
cur_day

2

In [6]:
# that's monitoring stuff - daily stats
def give_EI_feedback(EI_dir, current_day):
    print("Regarding Exercise Intensity...")
    for key,value in EI_dir.items():
        # for every EI, count how many days passed since 
        # the last time you had exercise of that intensity
        difference = current_day - value
        print(f"EI level '{key}': last exercised", end=" ")
        # handle negative values (no data for this particular
        # month), 0 (exercised today) etc
        if difference < 0:
            print("not this month")
        elif difference == 0:
            print("today")
        elif difference == 1:
            print("1 day ago")
        elif difference > 1:
            print(f"{difference} days ago")
            
give_EI_feedback(EI_last, cur_day)

Regarding Exercise Intensity...
EI level '1': last exercised not this month
EI level '2': last exercised 1 day ago
EI level '3': last exercised not this month
EI level '4': last exercised not this month
EI level '5': last exercised not this month


In [7]:
# let's think how to count how many time I had exercise
# of intensity level 3+ and how to present it in output
print("This week (last day - 7)")

This week (last day - 7)


In [8]:
df["Day"]

0    1
1    2
Name: Day, dtype: int64

In [9]:
df[["Day", "EI"]].tail(10)

,Day,EI
0,1,2
1,2,0


In [10]:
df.dtypes

Day       int64
DKind    object
Math      int64
CS        int64
Eng       int64
Total     int64
EC        int64
EI        int64
OD        int64
LE        int64
JG        int64
GM        int64
dtype: object

In [11]:
month_year_list = month.split()
month_alone = month_year_list[0]
year_alone = month_year_list[1]
month = month_alone
month

'April'

In [12]:
# Here we get the last day of recorded data and try to
# get a week interval by subracting 7 from the last day
last_week_day = df["Day"].iloc[-1]
first_week_day = last_week_day - 7
print(first_week_day)
# case of the first day of the month
if first_week_day == -6:
    print(f"On Apr 1, you exercised {EI3plus_count} times")
if first_week_day < 0:
    print("The new month hasn't accumulated seven plus days yet")

-5
The new month hasn't accumulated seven plus days yet


In [13]:
print(last_week_day)

2


In [14]:
last_week_day % 7

2

In [15]:
21 % 7

0

In [16]:
def get_EIL3plus_count(df, first_day):
    """
    Summary:
        Here we select days after the first day of 
        the interval AND days when EIL3+ exercises
        happened. Then we count how many rows of df
        meet the above requirements
   
        Note: EIL3+ stands for
        Exercise Intensity Level 3 (+) plus
    Args:
        df: df with 'Day' and 'EI' cols
    Returns:
        count of rows
    """
    return len(
        df.loc[(df["Day"] > first_day) & (df["EI"] >= 3)])

In [25]:
df.loc[df["Day"] > first_week_day]

,Day,DKind,Math,CS,Eng,Total,EC,EI,OD,LE,JG,GM
0,1,work,0,212,11,223,1,2,120,0,0,0
1,2,work,90,0,9,99,1,0,12,0,15,0


In [18]:
# df.loc[(df['column_name'] >= A) & (df['column_name'] <= B)]
df.loc[(df["Day"] > first_week_day) & (df["EI"] >= 3)]

,Day,DKind,Math,CS,Eng,Total,EC,EI,OD,LE,JG,GM


In [19]:
week_df = df.loc[df["Day"] > first_week_day]
week_df

,Day,DKind,Math,CS,Eng,Total,EC,EI,OD,LE,JG,GM
0,1,work,0,212,11,223,1,2,120,0,0,0
1,2,work,90,0,9,99,1,0,12,0,15,0


In [20]:
week_df.loc[week_df["EI"] >= 3]

,Day,DKind,Math,CS,Eng,Total,EC,EI,OD,LE,JG,GM


In [21]:
# think of renaming this var
EI3plus_count = len(week_df.loc[week_df["EI"] >= 3])
EI3plus_count

0

In [22]:
print(f"From Mar {first_week_day} to Mar {last_week_day}", end=", ")
print(f"you had EIL3+ exercises", end=" ")
if EI3plus_count == 1:
    print("only once")
else:
    print(f"{EI3plus_count} times")

From Mar -5 to Mar 2, you had EIL3+ exercises 0 times


In [23]:
if cur_week_EI_count < 2:
    print("You haven't exercised enough this week!")
else:
    print()

NameError: name 'cur_week_EI_count' is not defined

In [ ]:
# maybe this func is gonna belong to 'monitoring' package
def sport_stats():
    """
    (Example)
    You exercised this month: 10 times
    Last time you got medium exercise: 3 days ago
    Last time you got hard exercise: 7 days ago
    """
    pass